In [ ]:
from moviepy.editor import VideoFileClip, concatenate_videoclips
from natsort import natsorted
import os
import random  

def process_chunk(chunk, ad_clip_path, output_filename, chunk_size=20, ad_position=10):
    """
    Combine the videos in the chunk, insert the advertisement at the specified position,
    and save the output to a file.
    """
    ad_clip = VideoFileClip(ad_clip_path)  # Load the advertisement clip
    # Insert the advertisement at the specified position if the chunk is large enough
    if len(chunk) > ad_position:
        final_clips = chunk[:ad_position] + [ad_clip] + chunk[ad_position:]
    else:
        final_clips = chunk

    if final_clips:  # Check if there are any clips to process
        final_clip = concatenate_videoclips(final_clips)
        final_clip.write_videofile(output_filename, codec="libx264", fps=24)  # Adjust codec and fps as needed
        print(f"Output saved to {output_filename}")
    ad_clip.close()  # Close the ad clip to free up memory

# Path to the advertisements directory
ads_dir = r'G:\My Drive\Data_Science\Resulam\Phrasebook_Audio_Video_Processing_production\Ads_Video\ResulamVideosAds_Bon'

# Path to the chapters directories
chapters_dir = r'G:\My Drive\Data_Science\Resulam\Phrasebook_Audio_Video_Processing_production\Languages\NufiPhrasebook\bilingual_sentences'

# Loop through each chapter directory
for chapter in natsorted(os.listdir(chapters_dir)):
    chapter_path = os.path.join(chapters_dir, chapter)
    if os.path.isdir(chapter_path):
        print(f"Processing {chapter}...")
        
        # Naturally Sort Videos, and Take care of Special/Odd Videos containing the prefix "no_"
        all_videos = natsorted([v for v in os.listdir(chapter_path) if v.endswith(('.mp4', '.avi', '.mov'))])  # List all video files

        odd_sentences = [s for s in all_videos if "no_" in s]
        all_videos_to_sort = [s.split("no_",1)[-1] for s in all_videos]
        all_odd_sentences_suffix = [s.split("no_",1)[-1] for s in odd_sentences]
        all_videos_sorted = natsorted(all_videos_to_sort)
        all_videos_sorted = ["no_" + s if s in all_odd_sentences_suffix else s for s in all_videos_sorted]


        chunk_size = 20  # Set the chunk size
        ad_position = 10  # Set the ad position

        for i in range(0, len(all_videos_sorted), chunk_size):  # Process in chunks
            chunk = all_videos_sorted[i:i+chunk_size]
            video_clips = [VideoFileClip(os.path.join(chapter_path, video)) for video in chunk]

            # Choose a random advertisement video for each chunk
            ad_video = random.choice(os.listdir(ads_dir))
            ad_clip_path = os.path.join(ads_dir, ad_video)
            
            # Define the output filename for each chunk
            output_filename = os.path.join(chapters_dir, f"{chapter}_chunk_{i//chunk_size + 1}.mp4")
            
            # Process the current chunk with the randomly selected ad
            process_chunk(video_clips, ad_clip_path, output_filename, chunk_size, ad_position)

            # Close all clips in the current chunk to free up memory
            for clip in video_clips:
                clip.close()


Processing Chap1...


                                                                                                                       

t:   2%|█                                                              | 279/17101 [9:01:44<04:53, 57.41it/s, now=None]

                                                                                                                       

t:   2%|█                                                              | 279/17101 [9:01:44<04:53, 57.41it/s, now=None]

t:  59%|██████████████████████████████████████▍                          | 5220/8814 [39:32<04:54, 12.19it/s, now=None]

Moviepy - Building video G:\My Drive\Data_Science\Resulam\Phrasebook_Audio_Video_Processing_production\Languages\NufiPhrasebook\bilingual_sentences\Chap1_chunk_1.mp4.
MoviePy - Writing audio in Chap1_chunk_1TEMP_MPY_wvf_snd.mp3



chunk:  69%|█████████████████████████████████████████▌                  | 4593/6626 [00:08<00:03, 583.70it/s, now=None]


chunk:  99%|███████████████████████████████████████████████████████████▍| 6566/6626 [00:11<00:00, 749.45it/s, now=None]
                                                                                                                       

t:   2%|█                                                              | 279/17101 [9:01:56<04:53, 57.41it/s, now=None]

                                                                                                                       

t:   2%|█                                                              | 279/17101 [9:01:56<04:53, 57.41it/s, now=None]

t:  59%|██████████████████████████████████████▍                          | 5220/8814 [39:45<04:54, 12.19it/s, now=None]

...

t:  61%|███████████████████████████████████████▍                         | 4369/7212 [04:40<03:52, 12.23it/s, now=None]


t:  63%|████████████████████████████████████████▉                        | 4537/7212 [04:55<03:40, 12.14it/s, now=None]


t:  66%|██████████████████████████████████████████▋                      | 4743/7212 [05:18<02:24, 17.13it/s, now=None]


t:  69%|████████████████████████████████████████████▌                    | 4944/7212 [05:36<05:33,  6.81it/s, now=None]


t:  69%|████████████████████████████████████████████▊                    | 4971/7212 [05:38<02:49, 13.20it/s, now=None]

In [ ]:


# List all video files
all_videos = natsorted([v for v in os.listdir(chapter_path) if v.endswith(('.mp4', '.avi', '.mov'))])

# Identify odd sentences (videos with "no_" in their names)
odd_sentences = [s for s in all_videos if "no_" in s]

# Remove "no_" prefix for sorting
all_videos_to_sort = [s.split("no_", 1)[-1] if "no_" in s else s for s in all_videos]

# Sort video filenames naturally
all_videos_sorted = natsorted(all_videos_to_sort)

# Re-add "no_" prefix to odd sentences in the sorted list
all_videos_sorted_final = ["no_" + s if any(odd.endswith(s) for odd in odd_sentences) else s for s in all_videos_sorted]

print(odd_sentences)
print(all_videos_sorted_final)


In [23]:
# Naturally Sort Videos, and Take care of Special/Odd Videos containing the prefix "no_"
all_videos = natsorted([v for v in os.listdir(chapter_path) if v.endswith(('.mp4', '.avi', '.mov'))])  # List all video files

odd_sentences = [s for s in all_videos if "no_" in s]
all_videos_to_sort = [s.split("no_",1)[-1] for s in all_videos]
all_odd_sentences_suffix = [s.split("no_",1)[-1] for s in odd_sentences]
all_videos_sorted = natsorted(all_videos_to_sort)
all_videos_sorted = ["no_" + s if s in all_odd_sentences_suffix else s for s in all_videos_sorted]

all_videos_sorted

['no_english_nufi_phrasebook_260_padded.mp4']


['english_nufi_phrasebook_259_padded.mp4',
 'no_english_nufi_phrasebook_260_padded.mp4',
 'english_nufi_phrasebook_261_padded.mp4',
 'english_nufi_phrasebook_262_padded.mp4',
 'english_nufi_phrasebook_263_padded.mp4',
 'english_nufi_phrasebook_264_padded.mp4',
 'english_nufi_phrasebook_265_padded.mp4',
 'english_nufi_phrasebook_266_padded.mp4',
 'english_nufi_phrasebook_267_padded.mp4',
 'english_nufi_phrasebook_268_padded.mp4',
 'english_nufi_phrasebook_269_padded.mp4',
 'english_nufi_phrasebook_270_padded.mp4',
 'english_nufi_phrasebook_271_padded.mp4',
 'english_nufi_phrasebook_272_padded.mp4',
 'english_nufi_phrasebook_273_padded.mp4',
 'english_nufi_phrasebook_274_padded.mp4',
 'english_nufi_phrasebook_275_padded.mp4',
 'english_nufi_phrasebook_276_padded.mp4',
 'english_nufi_phrasebook_277_padded.mp4',
 'english_nufi_phrasebook_278_padded.mp4',
 'english_nufi_phrasebook_279_padded.mp4',
 'english_nufi_phrasebook_280_padded.mp4',
 'english_nufi_phrasebook_281_padded.mp4',
 'englis

['english_nufi_phrasebook_259_padded.mp4',
 'no_english_nufi_phrasebook_260_padded.mp4',
 'english_nufi_phrasebook_261_padded.mp4',
 'english_nufi_phrasebook_262_padded.mp4',
 'english_nufi_phrasebook_263_padded.mp4',
 'english_nufi_phrasebook_264_padded.mp4',
 'english_nufi_phrasebook_265_padded.mp4',
 'english_nufi_phrasebook_266_padded.mp4',
 'english_nufi_phrasebook_267_padded.mp4',
 'english_nufi_phrasebook_268_padded.mp4',
 'english_nufi_phrasebook_269_padded.mp4',
 'english_nufi_phrasebook_270_padded.mp4',
 'english_nufi_phrasebook_271_padded.mp4',
 'english_nufi_phrasebook_272_padded.mp4',
 'english_nufi_phrasebook_273_padded.mp4',
 'english_nufi_phrasebook_274_padded.mp4',
 'english_nufi_phrasebook_275_padded.mp4',
 'english_nufi_phrasebook_276_padded.mp4',
 'english_nufi_phrasebook_277_padded.mp4',
 'english_nufi_phrasebook_278_padded.mp4',
 'english_nufi_phrasebook_279_padded.mp4',
 'english_nufi_phrasebook_280_padded.mp4',
 'english_nufi_phrasebook_281_padded.mp4',
 'englis